In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Parallel ADK Agent Demo

> **Note**: This notebook is for reference and educational purposes only. Not intended for production use.  
> **Questions?** mateuswagner at google.com

This notebook demonstrates how to build a **parallel agent** using Google's Agent Development Kit (ADK).

## What is a ParallelAgent?

A **ParallelAgent** is a workflow agent that executes multiple sub-agents **concurrently**, enabling:
- Faster execution by running independent tasks simultaneously
- Concurrent data retrieval from multiple sources
- Parallel processing of independent operations

**Tech Stack**: ADK with Gemini 3


---
## Parallel Agent Demo

This demo shows how to build a **flexible parallel agent system** where:

### Agent Specialization by Information Type
Each agent specializes in a **type of information** (not a specific product):
- **PriceAgent**: Looks up pricing for ANY product
- **DetailsAgent**: Looks up detailed descriptions for ANY product  
- **BasicInfoAgent**: Provides basic summaries for ANY product

### How It Works
1. **User asks** about one or more products (e.g., "Tell me about smartphone and headphones")
2. **ParallelAgent** runs all 3 specialists concurrently, each storing results in session state
3. **MergerAgent** reads all results from state and creates a consolidated report
4. **SequentialAgent** orchestrates the flow: parallel execution → then consolidation

## Why This Architecture is Required

The **SequentialAgent([ParallelAgent, MergerAgent])** pattern is necessary because:
1. **ParallelAgent** executes sub-agents concurrently but stores results in session state via `output_key` - it doesn't automatically consolidate
2. **MergerAgent** needs to read from session state using `{placeholders}`, but these variables must exist before the agent starts
3. **SequentialAgent** guarantees execution order: ParallelAgent populates state first, then MergerAgent reads and consolidates
4. Without SequentialAgent, the merger would try to read state variables that don't exist yet, causing a KeyError
5. This is the official ADK pattern for parallel execution + consolidation - attempting to use a single coordinator agent fails because it can't get a second turn after delegation


---

## Get started

In [ ]:
## Installation
# Requirements: Python 3.10+, virtual environment, Google Cloud SDK (`gcloud init`)

%pip install --upgrade --quiet 'google-adk' nbformat

# Restart kernel after installation

## Import libraries

In [1]:
# Standard library imports
import asyncio
import json
import os
import warnings
from typing import Any
from datetime import datetime

# Third-party imports
from IPython.display import Markdown, display

# Google ADK imports
from google.adk.agents import Agent, ParallelAgent, SequentialAgent
from google.adk.events import Event
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types

### CONFIGURATION SETTINGS

In [2]:
# Google Cloud Project Configuration
PROJECT_ID = "matt-demos"  # TODO: Set your Google Cloud project ID

# Default region for Vertex AI resources
LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "global")

# Set environment variables
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID
os.environ["GOOGLE_CLOUD_LOCATION"] = LOCATION
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "True"

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

## Build ADK agent

Build your application using ADK, including the Gemini model and custom tools that you define.

---


### Set agent tools

To start, set the tools that a customer support agent needs to do their job.

In [3]:
def get_product_details(product_name: str):
    """Gathers detailed descriptions about a product."""
    details = {
        "smartphone": "A cutting-edge smartphone with advanced camera features and lightning-fast processing.",
        "usb charger": "A super fast and light usb charger",
        "shoes": "High-performance running shoes designed for comfort, support, and speed.",
        "headphones": "Wireless headphones with advanced noise cancellation technology for immersive audio.",
        "speaker": "A voice-controlled smart speaker that plays music, sets alarms, and controls smart home devices.",
    }
    return details.get(product_name, "Product details not found.")


def get_product_price(product_name: str):
    """Gathers price information about a product."""
    prices = {
        "smartphone": 500,
        "usb charger": 10,
        "shoes": 100,
        "headphones": 50,
        "speaker": 80,
    }
    return prices.get(product_name, "Product price not found.")


def get_basic_info(product_name: str):
    """Gathers basic information about a product (category, brand, availability)."""
    basic_info = {
        "smartphone": {
            "category": "Electronics",
            "brand": "TechPro",
            "availability": "In Stock",
            "rating": "4.5/5"
        },
        "usb charger": {
            "category": "Electronics",
            "brand": "PowerMax",
            "availability": "In Stock",
            "rating": "4.7/5"
        },
        "shoes": {
            "category": "Footwear",
            "brand": "RunFast",
            "availability": "In Stock",
            "rating": "4.6/5"
        },
        "headphones": {
            "category": "Audio",
            "brand": "SoundWave",
            "availability": "In Stock",
            "rating": "4.8/5"
        },
        "speaker": {
            "category": "Smart Home",
            "brand": "VoiceHub",
            "availability": "Limited Stock",
            "rating": "4.4/5"
        },
    }
    
    info = basic_info.get(product_name)
    if info:
        return f"Category: {info['category']}, Brand: {info['brand']}, Availability: {info['availability']}, Rating: {info['rating']}"
    return "Basic information not found."

### Set Agent model

See the [Gemini model documentation](https://cloud.google.com/vertex-ai/generative-ai/docs/learn/models) for detailed performance benchmarks and pricing.

In [4]:
model = "gemini-3-pro-preview"

### Assemble the ParallelAgent

A **ParallelAgent** executes multiple sub-agents concurrently, which is ideal when:
- Tasks can be performed independently without dependencies
- You need to retrieve data from multiple sources simultaneously
- You want to speed up workflows by running agents in parallel

**Key characteristics:**
- Not powered by an LLM (deterministic execution)
- Sub-agents run independently with no automatic state sharing
- All sub-agents start running at approximately the same time

In [5]:
# Session identifiers for tracking agent interactions
app_name = "parallel_product_research_app"
user_id = "user_one"
session_id = "session_one"

# Specialized agents for different information types
# Each agent stores results in session state via output_key
price_agent = Agent(
    name="PriceAgent",
    model=model,
    description="Agent that researches product prices for any product requested",
    instruction="You are a pricing specialist. When asked about product pricing, use the get_product_price tool to look up prices for the requested products. Return ONLY the price data in this format: 'Product: $price'. Do not add extra commentary.",
    tools=[get_product_price],
    output_key="price_result"
)

details_agent = Agent(
    name="DetailsAgent",
    model=model,
    description="Agent that researches detailed product descriptions for any product requested",
    instruction="You are a product details specialist. When asked about product details, use the get_product_details tool to look up detailed descriptions for the requested products. Return ONLY the description text. Do not add extra commentary.",
    tools=[get_product_details],
    output_key="details_result"
)

basic_info_agent = Agent(
    name="BasicInfoAgent",
    model=model,
    description="Agent that provides basic product information for any product requested",
    instruction="You are a basic product information specialist. When asked about products, use the get_basic_info tool to gather category, brand, availability, and rating information. Return ONLY this data in a structured format. Do not add extra commentary.",
    tools=[get_basic_info],
    output_key="basic_info_result"
)

# ParallelAgent runs all sub-agents concurrently
parallel_agent = ParallelAgent(
    name="ParallelProductResearchAgent",
    description="Coordinates parallel product research across multiple specialized agents",
    sub_agents=[price_agent, details_agent, basic_info_agent],
)

# MergerAgent reads results from session state using placeholders
# Placeholders are populated after ParallelAgent completes
merger_agent = Agent(
    name="MergerAgent",
    model=model,
    description="Synthesizes product research findings into a consolidated report",
    instruction="""You are a synthesis specialist that combines product research findings.

You will receive three pieces of information from the session state:
- Price information: {price_result}
- Product details: {details_result}
- Basic information: {basic_info_result}

Your task is to combine these into a comprehensive, well-structured report.

**CRITICAL: Use ONLY the information provided in the three inputs above. Do not add external knowledge.**

Output Format:

## Consolidated Product Research Report

### Product Pricing
{price_result}

### Product Details
{details_result}

### Basic Product Information
{basic_info_result}

### Summary
(Provide a brief 1-2 sentence synthesis connecting the findings above)

Output ONLY the structured report. Do not include any preamble or additional commentary.""",
)

# SequentialAgent ensures ParallelAgent completes before MergerAgent starts
# This guarantees session state is populated before merger reads it
root_agent = SequentialAgent(
    name="ResearchAndSynthesisPipeline",
    sub_agents=[parallel_agent, merger_agent],
    description="Coordinates parallel product research and synthesizes results into a final report"
)

In [6]:
async def run_parallel_agent_demo(user_query: str = None):
    """
    Executes the parallel agent demo and returns all events.
    
    Args:
        user_query: Product research request (default: smartphone and headphones query)
    
    Returns:
        List of events from the agent execution
    """
    # Initialize session service
    session_service = InMemorySessionService()
    
    # Create the session
    await session_service.create_session(
        app_name=app_name,
        user_id=user_id,
        session_id=session_id
    )
    
    # Create runner with the root agent
    runner = Runner(
        agent=root_agent,
        app_name=app_name,
        session_service=session_service,
    )
    
    # Use default query if none provided
    if user_query is None:
        user_query = "I need information about smartphone and headphones"
    
    # Create message content object
    message = types.Content(
        role="user",
        parts=[types.Part(text=user_query)]
    )
    
    # Execute the agent and collect events
    events = [event async for event in runner.run_async(
        user_id=user_id,
        session_id=session_id,
        new_message=message
    )]
    
    return events

In [7]:
# Execute the parallel agent demo
events = await run_parallel_agent_demo()

# Display the final consolidated report from MergerAgent
print("\n" + "="*80)
print("FINAL CONSOLIDATED REPORT")
print("="*80 + "\n")

merger_final_answer = None
# Search from the end to find the MergerAgent's response
for event in reversed(events):
    if (hasattr(event, 'author') and event.author == 'MergerAgent' and
        hasattr(event, 'content') and event.content and
        hasattr(event.content, 'parts')):
        for part in event.content.parts:
            if hasattr(part, 'text') and part.text:
                merger_final_answer = part.text
                break
        if merger_final_answer:
            break

if merger_final_answer:
    display(Markdown(merger_final_answer))
else:
    print("WARNING: No final answer from MergerAgent found")
    print("\nShowing all events for debugging:\n")

    for i, event in enumerate(events, 1):
        # Extract and format timestamp
        timestamp = getattr(event, 'timestamp', None) or getattr(event, 'created_at', None) or getattr(event, 'time', None)
        if timestamp:
            from datetime import datetime
            if isinstance(timestamp, (int, float)):
                time_str = datetime.fromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S')
            else:
                time_str = str(timestamp)
        else:
            time_str = 'N/A'

        print(f"\n{'='*80}")
        print(f"EVENT {i}/{len(events)} - Author: {getattr(event, 'author', 'Unknown')} - Time: {time_str}")
        print(f"{'='*80}")

        if hasattr(event, 'content') and event.content and hasattr(event.content, 'parts'):
            print(f"  Role: {getattr(event.content, 'role', 'N/A')}")
            for j, part in enumerate(event.content.parts, 1):
                if hasattr(part, 'text') and part.text:
                    print(f"    Text: {part.text[:300]}...")
                if hasattr(part, 'function_call') and part.function_call:
                    print(f"    Function Call: {part.function_call.name}")
                if hasattr(part, 'function_response') and part.function_response:
                    print(f"    Function Response: {part.function_response.name}")


FINAL CONSOLIDATED REPORT



## Consolidated Product Research Report

### Product Pricing
Smartphone: $500
Headphones: $50

### Product Details
A cutting-edge smartphone with advanced camera features and lightning-fast processing.

Wireless headphones with advanced noise cancellation technology for immersive audio.

### Basic Product Information
**Product: Smartphone**
*   **Category:** Electronics
*   **Brand:** TechPro
*   **Availability:** In Stock
*   **Rating:** 4.5/5

**Product: Headphones**
*   **Category:** Audio
*   **Brand:** SoundWave
*   **Availability:** In Stock
*   **Rating:** 4.8/5

### Summary
The TechPro Smartphone ($500) features advanced processing and cameras, while the SoundWave Headphones ($50) offer noise-canceling audio capabilities. Both products are highly rated and currently available in stock.

In [8]:
# Display event timeline with timestamps
print("ALL EVENTS TIMELINE")
print("-"*80 + "\n")

for i, event in enumerate(events, 1):
    # Extract timestamp from event
    timestamp = getattr(event, 'timestamp', None) or getattr(event, 'created_at', None) or getattr(event, 'time', None)
    
    # Format timestamp
    if timestamp:
        if isinstance(timestamp, (int, float)):
            time_str = datetime.fromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S.%f')[:-3]
        else:
            time_str = str(timestamp)
    else:
        time_str = 'N/A'
    
    # Get author
    author = getattr(event, 'author', 'Unknown')
    
    # Get event type summary
    event_summary = ""
    if hasattr(event, 'content') and event.content and hasattr(event.content, 'parts'):
        role = getattr(event.content, 'role', 'N/A')
        for part in event.content.parts:
            if hasattr(part, 'text') and part.text:
                event_summary = f"Text ({len(part.text)} chars)"
                break
            elif hasattr(part, 'function_call') and part.function_call:
                event_summary = f"FunctionCall: {part.function_call.name}"
                break
            elif hasattr(part, 'function_response') and part.function_response:
                event_summary = f"FunctionResponse: {part.function_response.name}"
                break
    
    print(f"[{i:2d}] {time_str} | {author:25s} | {event_summary}")


ALL EVENTS TIMELINE
--------------------------------------------------------------------------------

[ 1] 2025-12-09 14:57:22.584 | PriceAgent                | FunctionCall: get_product_price
[ 2] 2025-12-09 14:57:31.485 | PriceAgent                | FunctionResponse: get_product_price
[ 3] 2025-12-09 14:57:22.773 | BasicInfoAgent            | FunctionCall: get_basic_info
[ 4] 2025-12-09 14:57:32.916 | BasicInfoAgent            | FunctionResponse: get_basic_info
[ 5] 2025-12-09 14:57:31.487 | PriceAgent                | Text (32 chars)
[ 6] 2025-12-09 14:57:32.917 | BasicInfoAgent            | Text (256 chars)
[ 7] 2025-12-09 14:57:22.682 | DetailsAgent              | FunctionCall: get_product_details
[ 8] 2025-12-09 14:57:52.910 | DetailsAgent              | FunctionResponse: get_product_details
[ 9] 2025-12-09 14:57:52.911 | DetailsAgent              | Text (172 chars)
[10] 2025-12-09 14:57:55.280 | MergerAgent               | Text (804 chars)


---

# Thank you